# FPP multi objective optimization

### Setup distributed computing environment

See more details here: https://fuse.help/parallel.html

In [ ]:
if gethostname() == "saga.cluster"
    nodes = 4
    np = 30 * nodes
    using Pkg
    Pkg.activate("..")
    using Distributed
    using ClusterManagers
    if nprocs() < np
        addprocs(SlurmManager(np-nprocs()+1),exclusive="", topology=:master_worker)
    end
else
    using Distributed
    np = 4
    if nprocs() < np
        addprocs(np-nprocs()+1, topology=:master_worker)
    end
end
println("Working with $(nprocs()) processes")

### Import packages

**NOTE:** Import the `plotlyjs()` plotting backend instead of the usual gr() to interactively look at results in 3D

In [ ]:
using Revise
using FUSE
using Plots; gr();
global_logger(FUSE.logger);

### Get `ini` and `act` for FPP case and custmize as needed

In [ ]:
ini, act = FUSE.case_parameters(:FPP; version=:v1_demount, init_from=:scalars)
act.ActorTauenn.transport_model = :h98y2
act.ActorPFcoilsOpt.optimization_scheme = :none; # don't spend time optimizing the PFs

### As a good practice, test the actor/workflow that you want to optimize first

In [ ]:
dd = FUSE.init(ini, act)
FUSE.ActorWholeFacility(dd, act);

In [ ]:
# define the optimization objectives
OFL = FUSE.ObjectivesFunctionsLibrary
objective_functions = [OFL[:max_fusion], OFL[:min_cost], OFL[:max_flattop]]

# option to resume an optimization where it was left off
if true
    continue_results = missing
else
    continue_results = results
end

# define optimization parameters
# For real optimization studies the population size (N) and number of iterations should be bigger
# eg. N=1000, iterations=100
optimization_parameters = Dict(
    :N => 4,
    :iterations => 10,
    :continue_results => continue_results)

# run optimization
results = FUSE.workflow_multiobjective_optimization(ini, act, FUSE.ActorWholeFacility, objective_functions; optimization_parameters...);

### Plot multi-objective optimization results

In [ ]:
plot(results, [1, 2, 3]; design_space=false)

### How to: Define and use a custom FUSE workflow

In [ ]:
# Here `@everywhere` is needed to make all processes aware of the custom function
@everywhere function workflow_custom(ini,act)
    FUSE.init(dd, ini, act)
    FUSE.ActorEquilibriumTransport(dd, act)
    FUSE.ActorCXbuild(dd, act)
    return dd
end

# results = FUSE.workflow_multiobjective_optimization(ini, act, custom_workflow, objective_functions; optimization_parameters...);